In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Wed Jun 30 16:01:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:3B:00.0 Off |                  N/A |
| 28%   43C    P2   138W / 250W |   6319MiB / 12066MiB |     18%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN V             Off  | 00000000:5E:00.0 Off |                  N/A |
| 29%   

In [3]:
import os
os.chdir('/home/l/liny/ruofan/pytorch-metric-learning/src')
os.environ["CUDA_VISIBLE_DEVICES"]="1, 0"

In [4]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from torchvision import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

from pytorch_metric_learning.models import bninception
from pytorch_metric_learning import samplers
from pytorch_metric_learning.datasets.Cars196 import *

In [5]:
device = torch.device("cuda")

train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 64
num_epochs = 200
result_dir = './log'
exp_name = 'car196'
os.makedirs(result_dir, exist_ok=True)

In [26]:
train_data = Cars196MetricLearning(root='.', download=True, train=True, transform=train_transform)

test_data = Cars196MetricLearning(root='.', download=True, train=False, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./devkit/cars_test_annos_withlabels.mat
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./devkit/cars_test_annos_withlabels.mat


In [10]:
# sampler = samplers.MPerClassSampler(train_data.labels, 
#                                     m=5, 
#                                     length_before_new_iter=100000)

In [31]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
# train_loader = train_data.loader
# test_loader = test_data.loader()

AttributeError: 'Cars196MetricLearning' object has no attribute 'samples'

In [28]:
train_data.loader

<function torchvision.datasets.folder.default_loader(path)>

In [24]:
data

<module 'torch.utils.data' from '/home/l/liny/anaconda3/envs/mypy37/lib/python3.7/site-packages/torch/utils/data/__init__.py'>

In [19]:
model = bninception(dim=512, pretrained=None)
model = torch.nn.DataParallel(model).to(device)

In [20]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ### 
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print("Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(epoch, batch_idx, loss, mining_func.num_triplets))

### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, 
                                                  train_embeddings,
                                                  test_labels,
                                                  train_labels,
                                                  False)
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies["precision_at_1"]

In [21]:
### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.SoftTripleLoss(num_classes=2340, 
                                  embedding_size=512, 
                                  centers_per_class=5, 
                                  la=20, 
                                  gamma=0.1, 
                                  margin=0.01).to(device)

loss_optimizer = torch.optim.Adam([{"params": model.parameters(), "lr": 1e-4},
                                   {"params": loss_func.parameters(), "lr": 1e-2}])

mining_func = miners.TripletMarginMiner(margin = 0.2, distance = distance, type_of_triplets = "semi-hard")
accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)

In [22]:
### pytorch-metric-learning stuff ###

for epoch in range(1, num_epochs+1):
    train(model, loss_func, mining_func, device, train_loader, loss_optimizer, epoch)
    knn_acc = test(train_data, test_data, model, accuracy_calculator)
    if epoch % 50 == 0 or epoch == 1:
        torch.save(model.state_dict(), os.path.join(result_dir, '{}_epoch{}_knnAcc{:.4f}.pt'.format(exp_name, epoch, knn_acc)))


TypeError: 'function' object is not iterable